# XGBoost Regression – Untuned (GPU Compatible)

This notebook trains two **XGBoost regression models**:
- One using **all high-variance features** (`VarianceThreshold`)
- One using the **top 30 features** selected with `RandomForestRegressor`

The objective is to provide a strong baseline without any hyperparameter tuning, but with GPU acceleration.

In [1]:
import xgboost as xgb

print("XGBoost version:", xgb.__version__)

try:
    booster = xgb.Booster(params={'tree_method':'gpu_hist'})
    print("GPU support detected: tree_method='gpu_hist' works.")
except Exception as e:
    print("GPU support NOT detected:", e)

try:
    booster = xgb.Booster(params={'tree_method':'hist', 'device':'cuda'})
    print("GPU support detected: device='cuda' works.")
except Exception as e:
    print("GPU support NOT detected:", e)


XGBoost version: 2.0.3
GPU support detected: tree_method='gpu_hist' works.
GPU support detected: device='cuda' works.


In [2]:
import sys, os
# Add the project root to the Python path
project_root = os.path.abspath("../..")
sys.path.append(project_root)

import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold

from utils.constants import ML_READY_DATA_FILE, TEST_MODE
from utils.data_loader import DataLoader
from utils.train_test_metrics_logger import TrainTestMetricsLogger

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Display current test mode status
if TEST_MODE:
    print("TEST_MODE is ON – reduced data and iterations.")
else:
    print("TEST_MODE is OFF – full training.")

# === Load and prepare data ===
loader = DataLoader(ML_READY_DATA_FILE)
df = loader.load_data()
X = df.drop(columns=["price"])
y = df["price"]

# === Feature selection using variance threshold ===
selector = VarianceThreshold(threshold=0.01)
selector.fit(X)
X_reduced = X.loc[:, selector.get_support()]

# === Select top 30 features using Random Forest importance ===
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_reduced, y)
importances = rf.feature_importances_
top_features = pd.Series(importances, index=X_reduced.columns).sort_values(ascending=False).head(30).index.tolist()
X_top = X_reduced[top_features]

# === Split data for all features and top features ===
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)
X_train_top, X_test_top, _, _ = train_test_split(X_top, y, test_size=0.2, random_state=42)

# === Define XGBoost parameters ===
use_gpu = True
params = {
    "objective": "reg:squarederror",
    "random_state": 42,
    "n_estimators": 100 if TEST_MODE else 400,
    "tree_method": "hist",
    "device": "cuda" if use_gpu else "cpu",
    "n_jobs": -1,
    "verbosity": 2
}

# === Train XGBoost on all features ===
model_all = xgb.XGBRegressor(**params)
model_all.fit(X_train, y_train)
y_pred_train_all = model_all.predict(X_train)
y_pred_test_all = model_all.predict(X_test)

# === Train XGBoost on top 30 features ===
model_top = xgb.XGBRegressor(**params)
model_top.fit(X_train_top, y_train)
y_pred_train_top = model_top.predict(X_train_top)
y_pred_test_top = model_top.predict(X_test_top)

# === Define evaluation function ===
def evaluate(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

# === Evaluate performance ===
mae_train_all, rmse_train_all, r2_train_all = evaluate(y_train, y_pred_train_all)
mae_test_all, rmse_test_all, r2_test_all = evaluate(y_test, y_pred_test_all)

mae_train_top, rmse_train_top, r2_train_top = evaluate(y_train, y_pred_train_top)
mae_test_top, rmse_test_top, r2_test_top = evaluate(y_test, y_pred_test_top)

# === Initialize logger and log results ===
logger = TrainTestMetricsLogger()

logger.log(
    model_name=f"XGBoost CV (All Features){' [TEST]' if TEST_MODE else ''}",
    experiment_name=f"XGBoost Untuned (All Features){' [TEST]' if TEST_MODE else ''}",
    mae_train=mae_train_all,
    rmse_train=rmse_train_all,
    r2_train=r2_train_all,
    mae_test=mae_test_all,
    rmse_test=rmse_test_all,
    r2_test=r2_test_all,
    data_file=ML_READY_DATA_FILE,
    n_features=X_train.shape[1]  
)

logger.log(
    model_name=f"XGBoost CV (Top RF Features){' [TEST]' if TEST_MODE else ''}",
    experiment_name=f"XGBoost Untuned (Top RF Features){' [TEST]' if TEST_MODE else ''}",
    mae_train=mae_train_top,
    rmse_train=rmse_train_top,
    r2_train=r2_train_top,
    mae_test=mae_test_top,
    rmse_test=rmse_test_top,
    r2_test=r2_test_top,
    data_file=ML_READY_DATA_FILE,
    n_features=X_train_top.shape[1]  
)

# === Display summary table ===
logger.display_table()


TEST_MODE is OFF – full training.


Rank,Best,timestamp,model,mae_train,rmse_train,r2_train,mae_test,rmse_test,r2_test,r2_gap,r2_gap_diagnostic,n_features,interpretation,ranking_score
1,✔,2025-07-01 22:24:13,XGBoost CV (Top RF Features),22.1 k€,31.4 k€,0.977413,63.7 k€,96.3 k€,0.787667,0.189746,❌ Strong overfitting,30,overfitting,-159934.167991
2,,2025-07-01 22:24:13,XGBoost CV (All Features),20.3 k€,28.7 k€,0.981173,64.0 k€,96.2 k€,0.787700,0.193473,❌ Strong overfitting,72,overfitting,-160233.170278
3,,2025-07-01 22:23:31,XGBoost CV (All Features) [Fine-Tuned v6],55.0 k€,77.5 k€,0.863050,67.1 k€,98.6 k€,0.777142,0.085908,⚠️ Moderate overfitting,72,overfitting,-165692.175438
4,,2025-07-01 22:23:31,XGBoost CV (Top RF Features) [Fine-Tuned v6],56.7 k€,79.8 k€,0.854707,68.4 k€,100.3 k€,0.769638,0.085069,⚠️ Moderate overfitting,30,overfitting,-168652.811846


In [1]:
import sys, os
# Add the project root to the Python path
project_root = os.path.abspath("../..")
sys.path.append(project_root)

import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold

from utils.constants import ML_READY_DATA_FILE, TEST_MODE
from utils.data_loader import DataLoader
from utils.train_test_metrics_logger import TrainTestMetricsLogger

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

if TEST_MODE:
    print("TEST_MODE is ON – reduced data and iterations.")
else:
    print("TEST_MODE is OFF – full training.")

# === Load and prepare data ===
loader = DataLoader(ML_READY_DATA_FILE)
df = loader.load_data()
X = df.drop(columns=["price"])
y = df["price"]

# === Feature selection using variance threshold ===
selector = VarianceThreshold(threshold=0.01)
selector.fit(X)
X_reduced = X.loc[:, selector.get_support()]

# === Top 30 features via Random Forest ===
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_reduced, y)
top_features = pd.Series(rf.feature_importances_, index=X_reduced.columns).nlargest(30).index.tolist()
X_top = X_reduced[top_features]

# === Split data ===
X_dev_all, X_test_all, y_dev_all, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)
X_train_all, X_val_all, y_train_all, y_val_all = train_test_split(X_dev_all, y_dev_all, test_size=0.2, random_state=42)

X_dev_top, X_test_top, _, _ = train_test_split(X_top, y, test_size=0.2, random_state=42)
X_train_top, X_val_top, y_train_top, y_val_top = train_test_split(X_dev_top, y_dev_all, test_size=0.2, random_state=42)

# === Fine-tuned XGBoost params ===
use_gpu = True
params = {
    "objective": "reg:squarederror",
    "random_state": 42,
    "n_estimators": 100 if TEST_MODE else 600,
    "max_depth": 4,
    "min_child_weight": 6,
    "gamma": 0.4,
    "subsample": 0.6,
    "colsample_bytree": 0.6,
    "reg_alpha": 4.0,
    "reg_lambda": 5.0,
    "tree_method": "hist",
    "device": "cuda" if use_gpu else "cpu",
    "n_jobs": -1,
    "verbosity": 1
}

early_stopping = 10 if TEST_MODE else 50

# === Train on all features ===
model_all = xgb.XGBRegressor(**params)
model_all.fit(
    X_train_all, y_train_all,
    eval_set=[(X_val_all, y_val_all)],
    early_stopping_rounds=early_stopping,
    verbose=False
)
y_pred_train_all = model_all.predict(X_train_all)
y_pred_test_all = model_all.predict(X_test_all)

# === Train on top RF features ===
model_top = xgb.XGBRegressor(**params)
model_top.fit(
    X_train_top, y_train_top,
    eval_set=[(X_val_top, y_val_top)],
    early_stopping_rounds=early_stopping,
    verbose=False
)
y_pred_train_top = model_top.predict(X_train_top)
y_pred_test_top = model_top.predict(X_test_top)

# === Evaluation ===
def evaluate(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

mae_train_all, rmse_train_all, r2_train_all = evaluate(y_train_all, y_pred_train_all)
mae_test_all, rmse_test_all, r2_test_all = evaluate(y_test, y_pred_test_all)

mae_train_top, rmse_train_top, r2_train_top = evaluate(y_train_top, y_pred_train_top)
mae_test_top, rmse_test_top, r2_test_top = evaluate(y_test, y_pred_test_top)

# === Log results ===
logger = TrainTestMetricsLogger()

logger.log(
    model_name=f"XGBoost CV (All Features) [Fine-Tuned v6]{' [TEST]' if TEST_MODE else ''}",
    experiment_name=f"XGBoost FineTuned (All Features){' [TEST]' if TEST_MODE else ''}",
    mae_train=mae_train_all,
    rmse_train=rmse_train_all,
    r2_train=r2_train_all,
    mae_test=mae_test_all,
    rmse_test=rmse_test_all,
    r2_test=r2_test_all,
    data_file=ML_READY_DATA_FILE,
    n_features=X_train_all.shape[1]
)

logger.log(
    model_name=f"XGBoost CV (Top RF Features) [Fine-Tuned v6]{' [TEST]' if TEST_MODE else ''}",
    experiment_name=f"XGBoost FineTuned (Top RF Features){' [TEST]' if TEST_MODE else ''}",
    mae_train=mae_train_top,
    rmse_train=rmse_train_top,
    r2_train=r2_train_top,
    mae_test=mae_test_top,
    rmse_test=rmse_test_top,
    r2_test=r2_test_top,
    data_file=ML_READY_DATA_FILE,
    n_features=X_train_top.shape[1]
)

logger.display_table()


TEST_MODE is OFF – full training.


e:\_SoftEng\_BeCode\real-estate-price-predictor\.venv\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
e:\_SoftEng\_BeCode\real-estate-price-predictor\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:01:25] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
e:\_SoftEng\_BeCo

Rank,Best,timestamp,model,mae_train,rmse_train,r2_train,mae_test,rmse_test,r2_test,r2_gap,r2_gap_diagnostic,n_features,"interpretation (r2,mae_gap)",ranking_score
1,✔,2025-07-01 23:01:26,XGBoost CV (All Features) [Fine-Tuned v6],55.0 k€,77.5 k€,0.863050,67.1 k€,98.6 k€,0.777142,0.085908,Moderate overfitting,72.000000,overfitting,-165692.175438
2,,2025-07-01 23:01:26,XGBoost CV (Top RF Features) [Fine-Tuned v6],56.7 k€,79.8 k€,0.854707,68.4 k€,100.3 k€,0.769638,0.085069,Moderate overfitting,30.000000,overfitting,-168652.811846
3,,2025-07-01 22:59:16,Random Forest (All Features),26.2 k€,39.1 k€,0.965067,68.3 k€,101.4 k€,0.764228,0.200838,Strong overfitting,72.000000,overfitting,-169750.741399
4,,2025-07-01 22:59:08,Linear Regression (All Features),63.7 k€,94.0 k€,0.797768,76.9 k€,109.9 k€,0.723019,0.074748,Good generalization,nan,overfitting,-186832.774354
5,,2025-07-01 22:59:33,Polynomial Regression (Degree 2),66.0 k€,94.4 k€,0.795941,79.1 k€,116.2 k€,0.690439,0.105501,Moderate overfitting,72.000000,good generalization,-195341.498785
6,,2025-07-01 22:58:57,Linear Regression (All Features) CV 5-Fold,84.9 k€,119.9 k€,0.670367,84.9 k€,119.9 k€,0.670350,0.000017,Excellent generalization,53.000000,good generalization,-204837.913326


# 🎯 XGBoost Regression with Optuna Hyperparameter Tuning

This notebook trains two XGBoost regression models on real estate data, with **hyperparameter tuning using Optuna**. It includes all stages from loading the data to model diagnostics.

## Data Preparation

- Load the cleaned ML-ready dataset from a CSV file using `DataLoader`.
- Drop the target variable `price` to separate `X` and `y`.
- Apply `VarianceThreshold` to remove low-variance features (threshold = 0.01).
- Use a `RandomForestRegressor` to rank feature importance.
- Select the **top 30 most important features** for one of the models.


## Hyperparameter Tuning (Optuna)

Define the function `tune_xgboost_with_optuna(...)` that:

- Runs an Optuna optimization loop.
- Evaluates model performance with **5-Fold Cross-Validation**.
- Minimizes the **Root Mean Squared Error (RMSE)**.

### Tuned Hyperparameters:

- `max_depth`
- `learning_rate`
- `n_estimators`
- `subsample`, `colsample_bytree`
- `reg_alpha`, `reg_lambda`
- `min_child_weight`, `gamma`



## Train Final Models

Two models are trained:

- One using **all filtered features**
- One using the **top 30 features**

Each is trained using the **best parameters** found by Optuna.

---

## Evaluation

Models are evaluated using:

- `MAE`: Mean Absolute Error  
- `RMSE`: Root Mean Squared Error  
- `R<sup>2</sup>`: Coefficient of determination  

Results are logged with `ExperimentTracker`.



## Diagnostics

- Summary tables displayed with `ModelEvaluator`
- Residuals & diagnostic plots from `ModelVisualizer`
- Optionally, **SHAP values** can be plotted to understand feature importance



## Test Mode (Optional)

When `TEST_MODE = True`, the pipeline uses:

- A smaller dataset  
- Fewer Optuna trials (`n_trials = 3`)  

To speed up execution and debugging.


In [2]:
import sys, os

# Add the project root to the Python path
project_root = os.path.abspath("../..")
sys.path.append(project_root)

import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold

from utils.constants import ML_READY_DATA_FILE, TEST_MODE
from utils.data_loader import DataLoader
from utils.train_test_metrics_logger import TrainTestMetricsLogger

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# === Load dataset ===
loader = DataLoader(ML_READY_DATA_FILE)
df = loader.load_data()

X = df.drop(columns=["price"])
y = df["price"]

# === Remove low variance features ===
selector = VarianceThreshold(threshold=0.01)
X_reduced = X.loc[:, selector.fit(X).get_support()]

# === Select top 30 important features using Random Forest ===
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_reduced, y)
top_features = pd.Series(rf_model.feature_importances_, index=X_reduced.columns).nlargest(30).index.tolist()
X_top = X_reduced[top_features]

# === Split datasets for training/testing ===
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(
    X_reduced, y, test_size=0.2, random_state=42
)
X_train_top, X_test_top, y_train_top, y_test_top = train_test_split(
    X_top, y, test_size=0.2, random_state=42
)

# === Optuna hyperparameter tuning function ===
use_gpu = True
random_state = 42
n_trials = 3 if TEST_MODE else 50

def tune_xgboost_with_optuna(X_data, y_data, n_trials):
    def objective(trial):
        params = {
            "max_depth": trial.suggest_int("max_depth", 2, 6),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "subsample": trial.suggest_float("subsample", 0.6, 0.9),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.9),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 5.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 5.0),
            "min_child_weight": trial.suggest_float("min_child_weight", 5, 20),
            "gamma": trial.suggest_float("gamma", 0, 5),
            "tree_method": "hist",
            "device": "cuda" if use_gpu else "cpu",
            "random_state": random_state,
            "objective": "reg:squarederror",
            "n_jobs": -1
        }
        model = xgb.XGBRegressor(**params)
        cv = KFold(n_splits=5, shuffle=True, random_state=random_state)
        scores = -cross_val_score(model, X_data, y_data, scoring="neg_root_mean_squared_error", cv=cv)
        return scores.mean()

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials)
    return study.best_params

# === Train and evaluate function ===
def train_and_evaluate(X_train_full, y_train_full, X_test, y_test, model_name, experiment_name):
    # Split X_train_full into sub-train and validation for early stopping
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full, test_size=0.2, random_state=42
    )

    # Tune with Optuna on X_train (not on X_val)
    best_params = tune_xgboost_with_optuna(X_train, y_train, n_trials)

    # Train with early stopping on validation set
    model = xgb.XGBRegressor(**best_params)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=50,
        verbose=False
    )

    # Evaluate on full train and test set
    y_pred_train = model.predict(X_train_full)
    y_pred_test = model.predict(X_test)

    mae_train = mean_absolute_error(y_train_full, y_pred_train)
    rmse_train = root_mean_squared_error(y_train_full, y_pred_train)
    r2_train = r2_score(y_train_full, y_pred_train)

    mae_test = mean_absolute_error(y_test, y_pred_test)
    rmse_test = root_mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)

    logger.log(
        model_name=model_name,
        experiment_name=experiment_name,
        mae_train=mae_train,
        rmse_train=rmse_train,
        r2_train=r2_train,
        mae_test=mae_test,
        rmse_test=rmse_test,
        r2_test=r2_test,
        data_file=ML_READY_DATA_FILE,
        n_features=X_train_full.shape[1]
    )

    print(f"{model_name} trained and logged.")
    print(f"Train R²: {r2_train:.4f}, Test R²: {r2_test:.4f}")

# === Initialize logger ===
logger = TrainTestMetricsLogger()

# === Run training and logging for all features ===
train_and_evaluate(
    X_train_all, y_train_all, X_test_all, y_test_all,
    model_name="XGBoost + Optuna CV + Early Stopping(All Features)",
    experiment_name="XGBoost + Optuna-Tuned (All Features)"
)

# === Run training and logging for top 30 features ===
train_and_evaluate(
    X_train_top, y_train_top, X_test_top, y_test_top,
    model_name="XGBoost + Optuna CV + Early Stopping(Top 30 Features)",
    experiment_name="XGBoost + Optuna-Tuned (Top 30 Features)"
)

# === Display logged results ===
logger.display_table(n_rows=10)


[I 2025-07-01 23:03:10,463] A new study created in memory with name: no-name-46f01f80-977b-4b6d-b8bc-00dadb34a9c2
[I 2025-07-01 23:03:12,199] Trial 0 finished with value: 105934.26419840306 and parameters: {'max_depth': 3, 'learning_rate': 0.1394927383082412, 'n_estimators': 121, 'subsample': 0.6526097024473726, 'colsample_bytree': 0.6439551539161069, 'reg_alpha': 3.3105221320379807, 'reg_lambda': 2.3620142698287556, 'min_child_weight': 7.966753546444872, 'gamma': 1.4302699459385644}. Best is trial 0 with value: 105934.26419840306.
[I 2025-07-01 23:03:16,626] Trial 1 finished with value: 98410.1995372556 and parameters: {'max_depth': 3, 'learning_rate': 0.1605337391634462, 'n_estimators': 349, 'subsample': 0.703814631273926, 'colsample_bytree': 0.6721285431799526, 'reg_alpha': 2.287965146533392, 'reg_lambda': 4.516068240740063, 'min_child_weight': 16.85066348505134, 'gamma': 4.12066634534115}. Best is trial 1 with value: 98410.1995372556.
[I 2025-07-01 23:03:22,498] Trial 2 finished wi

XGBoost + Optuna CV + Early Stopping(All Features) trained and logged.
Train R²: 0.8709, Test R²: 0.7950


[I 2025-07-01 23:08:49,002] Trial 0 finished with value: 98190.49364765952 and parameters: {'max_depth': 5, 'learning_rate': 0.09630813272186171, 'n_estimators': 319, 'subsample': 0.885285992667767, 'colsample_bytree': 0.7675267618490318, 'reg_alpha': 4.169328590307364, 'reg_lambda': 1.3116416927264287, 'min_child_weight': 13.448707342549802, 'gamma': 4.715109384803928}. Best is trial 0 with value: 98190.49364765952.
[I 2025-07-01 23:08:51,248] Trial 1 finished with value: 116930.09529305816 and parameters: {'max_depth': 2, 'learning_rate': 0.047125337292986264, 'n_estimators': 227, 'subsample': 0.6199355964603914, 'colsample_bytree': 0.7103920111533724, 'reg_alpha': 4.741103694460138, 'reg_lambda': 4.41428491390529, 'min_child_weight': 19.754080022077694, 'gamma': 3.2015623632223815}. Best is trial 0 with value: 98190.49364765952.
[I 2025-07-01 23:08:52,788] Trial 2 finished with value: 110006.34184468204 and parameters: {'max_depth': 2, 'learning_rate': 0.15215427704313547, 'n_estima

XGBoost + Optuna CV + Early Stopping(Top 30 Features) trained and logged.
Train R²: 0.8672, Test R²: 0.7900


Rank,Best,timestamp,model,mae_train,rmse_train,r2_train,mae_test,rmse_test,r2_test,r2_gap,r2_gap_diagnostic,n_features,"interpretation (r2,mae_gap)",ranking_score
1,✔,2025-07-01 23:08:43,XGBoost + Optuna CV + Early Stopping(All Features),51.5 k€,75.1 k€,0.870899,63.4 k€,94.6 k€,0.794986,0.075913,Good generalization,72.000000,overfitting,-157934.687697
2,,2025-07-01 23:14:00,XGBoost + Optuna CV + Early Stopping(Top 30 Features),52.4 k€,76.1 k€,0.867237,64.2 k€,95.7 k€,0.789978,0.077259,Good generalization,30.000000,overfitting,-159913.694661
3,,2025-07-01 23:01:26,XGBoost CV (All Features) [Fine-Tuned v6],55.0 k€,77.5 k€,0.863050,67.1 k€,98.6 k€,0.777142,0.085908,Moderate overfitting,72.000000,overfitting,-165692.175438
4,,2025-07-01 23:01:26,XGBoost CV (Top RF Features) [Fine-Tuned v6],56.7 k€,79.8 k€,0.854707,68.4 k€,100.3 k€,0.769638,0.085069,Moderate overfitting,30.000000,overfitting,-168652.811846
5,,2025-07-01 22:59:16,Random Forest (All Features),26.2 k€,39.1 k€,0.965067,68.3 k€,101.4 k€,0.764228,0.200838,Strong overfitting,72.000000,overfitting,-169750.741399
6,,2025-07-01 22:59:08,Linear Regression (All Features),63.7 k€,94.0 k€,0.797768,76.9 k€,109.9 k€,0.723019,0.074748,Good generalization,nan,overfitting,-186832.774354
7,,2025-07-01 22:59:33,Polynomial Regression (Degree 2),66.0 k€,94.4 k€,0.795941,79.1 k€,116.2 k€,0.690439,0.105501,Moderate overfitting,72.000000,good generalization,-195341.498785
8,,2025-07-01 22:58:57,Linear Regression (All Features) CV 5-Fold,84.9 k€,119.9 k€,0.670367,84.9 k€,119.9 k€,0.670350,0.000017,Excellent generalization,53.000000,good generalization,-204837.913326


# Saving XGBoost + Optuna Hyperparameter Tuning Models (`.pkl`) After Training

After training XGBoost models with Optuna tuning, it's essential to persist the trained models using `.pkl` files. The script below ensures each model is saved with a unique, timestamped filename and organized in the correct directory.


##  What the Script Does

1. **Appends the project root** to the Python path (to allow relative imports).
2. **Generates a timestamped filename**, including an optional `_TEST` suffix if `TEST_MODE` is enabled.
3. **Ensures the target directory exists**, and creates it if necessary.
4. **Saves both trained models** using `joblib.dump()`:
   - One trained with **all features**.
   - One trained with the **top 30 features** (e.g., selected via Random Forest).

In [4]:
import sys, os

# Add the project root to the Python path
project_root = os.path.abspath("../..")
sys.path.append(project_root)

import joblib
from datetime import datetime
from utils.constants import TEST_MODE, MODELS_DIR

# Create timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# Add suffix if in TEST mode
suffix = "_TEST" if TEST_MODE else ""

# Define subdirectory for .pkl files
PKL_DIR = os.path.join(MODELS_DIR, "pkl")
os.makedirs(PKL_DIR, exist_ok=True)

# Build filenames
filename_all = f"xgboost_optuna_all_{timestamp}{suffix}.pkl"
filename_top = f"xgboost_optuna_top30_{timestamp}{suffix}.pkl"

# Save models
joblib.dump(model_all, os.path.join(PKL_DIR, filename_all))
joblib.dump(model_top, os.path.join(PKL_DIR, filename_top))

print(f"[✔] Models saved to '{PKL_DIR}' as:\n - {filename_all}\n - {filename_top}")


[✔] Models saved to 'e:\_SoftEng\_BeCode\real-estate-price-predictor\models\pkl' as:
 - xgboost_optuna_all_20250701_1956.pkl
 - xgboost_optuna_top30_20250701_1956.pkl


# Saving Feature Lists Used by Each Model (`.json`)

After training and saving your machine learning models (e.g., XGBoost or CatBoost), it's critical to also save the **list of features** used during training. This ensures **inference compatibility** and prevents mismatches between the model and the input data.


## What the Script Does

1. **Creates the directory** for storing feature metadata:
   - Located in: `models/features/`

2. **Saves two JSON files**:
   - One listing the full set of features used in the **all-features model**.
   - One listing the selected **top 30 features** (e.g., based on feature importance).

3. **Uses the same base name as the corresponding `.pkl` model**, replacing the extension:
   - Example: `xgboost_optuna_all_20250629_1430.pkl` → `xgboost_optuna_all_20250629_1430.json`





In [5]:
import json

# Define subdirectory for features
FEATURES_DIR = os.path.join(MODELS_DIR, "features")
os.makedirs(FEATURES_DIR, exist_ok=True)

# Save features used for each model
with open(os.path.join(FEATURES_DIR, filename_all.replace(".pkl", ".json")), "w") as f:
    json.dump(list(X_reduced.columns), f, indent=2)

with open(os.path.join(FEATURES_DIR, filename_top.replace(".pkl", ".json")), "w") as f:
    json.dump(top_features, f, indent=2)

print(f"[✔] Associated feature files saved to '{FEATURES_DIR}'")


[✔] Associated feature files saved to 'e:\_SoftEng\_BeCode\real-estate-price-predictor\models\features'
